[![image](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/opengeos/maxar-open-data/blob/master/examples/morocco_earthquake.ipynb)
[![image](https://img.shields.io/badge/Open-Planetary%20Computer-black?style=flat&logo=microsoft)](https://pccompute.westeurope.cloudapp.azure.com/compute/hub/user-redirect/git-pull?repo=https://github.com/opengeos/maxar-open-data&urlpath=lab/tree/maxar-open-data/examples/morocco_earthquake&branch=master)
[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeos/maxar-open-data/blob/master/examples/morocco_earthquake.ipynb)
[![image](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/opengeos/maxar-open-data/master?urlpath=lab%2Ftree%2Fexamples)

**Visualizing Maxar Open Data for the 2023 Morocco Earthquake with Leafmap**

The [Maxar Open Data Program](https://www.maxar.com/open-data) provides pre- and post-event high-resolution satellite imagery in support of emergency planning, risk assessment, monitoring of staging areas and emergency response, damage assessment, and recovery. Check out the links below for more information.
- [Maxar Open Data Program](https://www.maxar.com/open-data)
- [Maxar Open Data on AWS](https://registry.opendata.aws/maxar-open-data/)
- [Maxar Open Data on STAC Index](https://stacindex.org/catalogs/maxar-open-data-catalog-ard-format#/)
- [Maxar Open Data on STAC Browser](https://radiantearth.github.io/stac-browser/#/external/maxar-opendata.s3.amazonaws.com/events/catalog.json?.language=en)

The Maxar Open Data STAC catalog URL is: https://maxar-opendata.s3.amazonaws.com/events/catalog.json


In [1]:
# !pip install -U leafmap geopandas cogeo-mosaic

In [2]:
import leafmap

Retrieve all collections from the Maxar Open Data STAC catalog. Each collection represents a single event.

In [3]:
leafmap.maxar_collections()

['BayofBengal-Cyclone-Mocha-May-23',
 'Emilia-Romagna-Italy-flooding-may23',
 'Gambia-flooding-8-11-2022',
 'Hurricane-Fiona-9-19-2022',
 'Hurricane-Ian-9-26-2022',
 'Hurricane-Idalia-Florida-Aug23',
 'Indonesia-Earthquake22',
 'Kahramanmaras-turkey-earthquake-23',
 'Kalehe-DRC-Flooding-5-8-23',
 'Marshall-Fire-21-Update',
 'Maui-Hawaii-fires-Aug-23',
 'McDougallCreekWildfire-BC-Canada-Aug-23',
 'Morocco-Earthquake-Sept-2023',
 'NWT-Canada-Aug-23',
 'New-Zealand-Flooding22',
 'New-Zealand-Flooding23',
 'Sudan-flooding-8-22-2022',
 'afghanistan-earthquake22',
 'cyclone-emnati22',
 'ghana-explosion22',
 'kentucky-flooding-7-29-2022',
 'pakistan-flooding22',
 'shovi-georgia-landslide-8Aug23',
 'southafrica-flooding22',
 'tonga-volcano21',
 'volcano-indonesia21',
 'yellowstone-flooding22']

Retrieve all collections for a specific event:

In [4]:
collections = leafmap.maxar_child_collections('Morocco-Earthquake-Sept-2023')
print(f"The number of collections: {len(collections)}")

The number of collections: 22


In [5]:
collections.sort()
collections

['10300100CB8C5600',
 '10300100CBAF7D00',
 '10300100E5404900',
 '10300100E5A0A100',
 '10300100EB8B4500',
 '10300100EBC1A000',
 '10300100ECC53700',
 '10300100ED11EA00',
 '10300500CFF91800',
 '10300500E4F91400',
 '10300500E4F91500',
 '10300500E4F91700',
 '10300500E4F91800',
 '10300500E4F91900',
 '10400100797DAC00',
 '1040010083289000',
 '10400100889ABF00',
 '104001008A8E9800',
 '104001008B75BB00',
 '1040050047DC6400',
 '1040050057DC8500',
 '1050050030DE8900']

Retrieve all items (tiles) for a specific collection and generate the footprints:

In [6]:
gdf = leafmap.maxar_items(
    collection_id='Morocco-Earthquake-Sept-2023', 
    child_id='10300100ECC53700', 
    return_gdf=True, 
    assets=['visual'])
gdf.head()

,geometry,datetime,platform,gsd,ard_metadata_version,catalog_id,utm_zone,quadkey,view:off_nadir,view:azimuth,...,view:sun_azimuth,view:sun_elevation,proj:epsg,proj:geometry,grid:code,proj:bbox,tile:data_area,tile:clouds_area,tile:clouds_percent,visual
0,"POLYGON ((-8.99837 30.61125, -8.99837 30.59612...",2023-09-10T11:46:24Z,WV02,0.83,0.0.1,10300100ECC53700,29,031313133131,41.5,234.2,...,154.6,62.3,32629,"{'type': 'Polygon', 'coordinates': [[[500155.9...",MXRA-Z29-031313133131,"497536.92626953125,3384843.75,500156.25,338652...",3.2,0.0,0,https://maxar-opendata.s3.amazonaws.com/events...
1,"POLYGON ((-9.02553 30.59894, -8.99837 30.59894...",2023-09-10T11:46:24Z,WV02,0.82,0.0.1,10300100ECC53700,29,031313133133,41.5,234.2,...,154.6,62.3,32629,"{'type': 'Polygon', 'coordinates': [[[497552.1...",MXRA-Z29-031313133133,"497552.18505859375,3379843.75,500156.25,338515...",13.6,0.0,0,https://maxar-opendata.s3.amazonaws.com/events...
2,"POLYGON ((-9.02489 30.55382, -8.99837 30.55383...",2023-09-10T11:46:25Z,WV02,0.82,0.0.1,10300100ECC53700,29,031313133311,41.4,234.4,...,154.6,62.3,32629,"{'type': 'Polygon', 'coordinates': [[[497612.6...",MXRA-Z29-031313133311,"497612.60986328125,3374843.75,500156.25,338015...",13.3,0.0,0,https://maxar-opendata.s3.amazonaws.com/events...
3,"POLYGON ((-9.02435 30.50870, -8.99837 30.50871...",2023-09-10T11:46:25Z,WV02,0.82,0.0.1,10300100ECC53700,29,031313133313,41.4,234.6,...,154.6,62.3,32629,"{'type': 'Polygon', 'coordinates': [[[497663.2...",MXRA-Z29-031313133313,"497663.26904296875,3369843.75,500156.25,337515...",13.1,0.0,0,https://maxar-opendata.s3.amazonaws.com/events...
4,"POLYGON ((-9.02396 30.46358, -8.99837 30.46359...",2023-09-10T11:46:25Z,WV02,0.82,0.0.1,10300100ECC53700,29,031313133331,41.4,234.8,...,154.6,62.4,32629,"{'type': 'Polygon', 'coordinates': [[[497699.8...",MXRA-Z29-031313133331,"497604.3701171875,3364843.75,500156.25,3370156.25",13.1,0.0,0,https://maxar-opendata.s3.amazonaws.com/events...


Add the footprints to the map:

In [7]:
m = leafmap.Map()
m.add_gdf(gdf, layer_name="Footprints", zoom_to_layer=True)
m

Retrieve the COG URLs for all tiles in a collection:

In [8]:
images = gdf['visual'].tolist()
images[:5]

['https://maxar-opendata.s3.amazonaws.com/events/Morocco-Earthquake-Sept-2023/ard/29/031313133131/2023-09-10/10300100ECC53700-visual.tif',
 'https://maxar-opendata.s3.amazonaws.com/events/Morocco-Earthquake-Sept-2023/ard/29/031313133133/2023-09-10/10300100ECC53700-visual.tif',
 'https://maxar-opendata.s3.amazonaws.com/events/Morocco-Earthquake-Sept-2023/ard/29/031313133311/2023-09-10/10300100ECC53700-visual.tif',
 'https://maxar-opendata.s3.amazonaws.com/events/Morocco-Earthquake-Sept-2023/ard/29/031313133313/2023-09-10/10300100ECC53700-visual.tif',
 'https://maxar-opendata.s3.amazonaws.com/events/Morocco-Earthquake-Sept-2023/ard/29/031313133331/2023-09-10/10300100ECC53700-visual.tif']

Download the COGs to a local directory:

In [ ]:
leafmap.maxar_download(images[:2])

Create a mosaic json file for the collection. You need to install `cogeo-mosaic` first using `pip install cogeo-mosaic`. Creating a mosaic json file might take a few minutes. Please be patient.

In [10]:
# leafmap.create_mosaicjson(images, output='10300100ECC53700.json')

Make the mosaic json file available on the web, then you can add the mosaic to the map:

In [11]:
m = leafmap.Map(height='600px')
m.add_basemap('SATELLITE')
url = 'https://open.gishub.org/maxar-open-data/datasets/Morocco-Earthquake-Sept-2023/10300100ECC53700.json'
m.add_stac_layer(url, name="Mosaic")
m.add_gdf(gdf, layer_name="Footprints")
m

Google Maps API key is required to use Google Maps. You can generate one from https://bit.ly/3sw0THG and use geemap.set_api_key(), defaulting to Esri basemaps.


Retrieve the footprint of all tiles for a specific event. This might take 15+ minutes. Please be patient.

In [12]:
# gdf = leafmap.maxar_all_items(
#     collection_id='Morocco-Earthquake-Sept-2023',
#     return_gdf=True, 
#     verbose=True
#     )
# gdf.to_file('maxar_footprints.geojson', driver='GeoJSON')
# gdf

Add the footprints to the map:

In [13]:
m = leafmap.Map(center=[31.35874, -8.78226], zoom=8)
url = 'https://open.gishub.org/maxar-open-data/datasets/Morocco-Earthquake-Sept-2023.geojson'
m.add_geojson(url, layer_name="Footprints")
m